In [1]:
import requests
import pandas as pd
d = {1686: 'BTCUSDT',
 1690: 'ETHUSDT',
 2189: 'DOGEUSDT',
 1692: 'LTCUSDT',
 1698: 'ADAUSDT',
 2209: 'SOLUSDT'}

In [3]:


# URL для запроса
url = "https://clientapi.bondah.com/articles"

# Заголовки запроса
headers = {
    "LanguageId": "7"  # Укажите нужный язык, например 'en' для английского
}

# Параметры запроса
params = {
    "FromId": None,           # ID статьи, после которой получить статьи (если не нужно, оставить None)
    "ArticlesCount": 3,      # Количество статей (по умолчанию 10)
    "InstrumentId": 1690,
    "IsSignals": True

}

# # Удаляем None из параметров, чтобы не отправлять их
# params = {k: v for k, v in params.items() if v is not None}

# Выполнение GET-запроса
try:
    response = requests.get(url, headers=headers, params=params)
    response.raise_for_status()  # Проверка на успешность запроса
    articles = response.json()  # Декодирование ответа в JSON
    print("Статьи получены:", articles)
except requests.exceptions.RequestException as e:
    print("Ошибка выполнения запроса:", e)

Статьи получены: [{'id': 24233389, 'instrumentId': 1690, 'analyseTypeId': 1, 'analyseType': 'Divergences', 'snapshotId': 97850, 'header': 'Ethereum M30: RSI сигнализирует о возможном развороте вверх', 'body': 'Цена не смогла достичь нового минимума, в то время как индикатор RSI показывает восходящую тенденцию. Образовавшаяся обратная дивергенция может сигнализировать о потенциальном развороте цены вверх.<br/>Гистограмма MACD находится ниже сигнальной линии. Этот индикатор показывает среднесрочные и долгосрочные тенденции и в некоторых случаях может игнорировать краткосрочные развороты и откаты цены.<br/>На индикаторе Stochastic голубая главная линия продолжает оставаться ниже сигнальной красной линии. Текущее состояние индикатора не отменяет и не противоречит сигналам, полученным от других технических индикаторов.<br/>Таким образом, по Ethereum рекомендуется сосредоточиться на открытии длинных позиций. Входить в рынок можно по текущим котировкам или при повторном тестировании уровня по

In [168]:
df = pd.DataFrame(articles)
df['createTime'] = pd.to_datetime(df['createTime'])
df.drop(['analyseTypeId', 'snapshotId', 'timeAgo', 'mtPicture'], axis=1, inplace=True)
df


,id,instrumentId,analyseType,header,body,createTime,foreCast
0,22415743,1690,Wolfe's Waves,Ethereum M15: Волны Вульфа указывают на возмож...,На графике виден очередной восходящий импульс....,2024-12-12 03:01:41,Down
1,22412393,1690,Divergences,Ethereum M15: RSI сигнализирует о возможном ра...,"Цена не смогла достичь нового минимума, в то в...",2024-12-12 02:01:22,Up
2,22362293,1690,Divergences,Ethereum M15: Дивергенция на снижение от RSI,"Цена достигла нового локального максимума, но ...",2024-12-11 20:45:31,Down


In [169]:
import re
def extract_trading_levels(text):
    # Шаблоны для поиска входа, целей и стоп-лосса
    patterns = {
        'entry': re.compile(r'\b(?:Вход|Entry):\s*(\d+\.\d+)'),
        'tp1': re.compile(r'\b(?:Первая цель|TP1):\s*(\d+\.\d+)'),
        'tp2': re.compile(r'\b(?:Вторая цель|TP2):\s*(\d+\.\d+)'),
        'sl': re.compile(r'\b(?:Стоп-ордер|SL):\s*(\d+\.\d+)')
    }

    # Результаты поиска
    levels = {}
    for key, pattern in patterns.items():
        match = pattern.search(text)
        levels[key] = float(match.group(1)) if match else None

    return levels


# Пример текста
text = '''Bitcoin сохраняет восходящую направленность движения, в то время как Stochastic опустился ниже 20. 
Образовавшаяся обратная дивергенция отмечена на графике зеленой линией. 
Перейдем к рассмотрению индикатора RSI. На данном осцилляторе разворотный сигнал был зафиксирован раньше. 
Формирование одновременно нескольких разворотных паттернов повышают вероятность отработки полученного сигнала.
Гистограмма MACD находится ниже сигнальной линии. Этот индикатор показывает среднесрочные и долгосрочные тенденции и в некоторых случаях может игнорировать краткосрочные развороты и откаты цены.
На данном таймфрейме по Bitcoin предпочтительнее открывать длинные позиции.

📌 Вход: 100808.80
✔️ Первая цель: 101812.20
✔️ Вторая цель: 103062.15
❌ Стоп-ордер: 100057.55
'''

# Пример использования
levels = extract_trading_levels(articles[1]['body'])
print(levels)


{'entry': 3803.48, 'tp1': 3846.94, 'tp2': 3892.01, 'sl': 3773.85}


In [170]:
# Преобразуем результат функции в DataFrame и добавляем его к df
df[['entry', 'tp1', 'tp2', 'sl']] = df['body'].apply(lambda x: pd.Series(extract_trading_levels(x)))

df

,id,instrumentId,analyseType,header,body,createTime,foreCast,entry,tp1,tp2,sl
0,22415743,1690,Wolfe's Waves,Ethereum M15: Волны Вульфа указывают на возмож...,На графике виден очередной восходящий импульс....,2024-12-12 03:01:41,Down,3879.36,3795.44,NaN,3897.13
1,22412393,1690,Divergences,Ethereum M15: RSI сигнализирует о возможном ра...,"Цена не смогла достичь нового минимума, в то в...",2024-12-12 02:01:22,Up,3803.48,3846.94,3892.01,3773.85
2,22362293,1690,Divergences,Ethereum M15: Дивергенция на снижение от RSI,"Цена достигла нового локального максимума, но ...",2024-12-11 20:45:31,Down,3828.09,3715.46,3655.41,3885.65


In [171]:
# Заменяем значения в колонке 'instrumentId' на соответствующие названия
df['instrumentId'] = df['instrumentId'].replace(d)
df['createTime'] = df['createTime'].astype(str)
df['is_check'] = 0
df

,id,instrumentId,analyseType,header,body,createTime,foreCast,entry,tp1,tp2,sl,is_check
0,22415743,ETHUSDT,Wolfe's Waves,Ethereum M15: Волны Вульфа указывают на возмож...,На графике виден очередной восходящий импульс....,2024-12-12 03:01:41,Down,3879.36,3795.44,NaN,3897.13,0
1,22412393,ETHUSDT,Divergences,Ethereum M15: RSI сигнализирует о возможном ра...,"Цена не смогла достичь нового минимума, в то в...",2024-12-12 02:01:22,Up,3803.48,3846.94,3892.01,3773.85,0
2,22362293,ETHUSDT,Divergences,Ethereum M15: Дивергенция на снижение от RSI,"Цена достигла нового локального максимума, но ...",2024-12-11 20:45:31,Down,3828.09,3715.46,3655.41,3885.65,0


In [172]:
import sqlite3
# Подключение к базе данных
conn = sqlite3.connect('trading_analysis.db')

In [173]:
df.to_sql('Trade_Signals', conn, if_exists='append', index=False)


IntegrityError: UNIQUE constraint failed: Trade_Signals.id

In [174]:
# Функция для проверки существования записи
def record_exists(cursor, record_id):
    cursor.execute("SELECT COUNT(*) FROM Trade_Signals WHERE id = ?", (record_id,))
    return cursor.fetchone()[0] > 0

# Вставка данных из DataFrame с проверкой
cursor = conn.cursor()
for index, row in df.iterrows():
    if not record_exists(cursor, row['id']):  # Проверяем, существует ли запись
        cursor.execute('''
        INSERT INTO Trade_Signals (
            id, instrumentId, analyseType, header, body, createTime, foreCast, entry, tp1, tp2, sl, is_check
        ) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
        ''', (
            row['id'], row['instrumentId'], row['analyseType'], row['header'], row['body'],
            row['createTime'], row['foreCast'], row['entry'], row['tp1'], row['tp2'], row['sl'], row['is_check']
        ))
    else:
        print(f"Record with id={row['id']} already exists. Skipping.")


# Сохранение изменений и закрытие соединения
conn.commit()
conn.close()

Record with id=22415743 already exists. Skipping.
Record with id=22412393 already exists. Skipping.
